In [1]:
import pandas as pd

dataframe = pd.read_csv("wggesucht_scraped_16_05.csv")

display(dataframe)

,Url,Title,Size,PriceTotal,Price,ExtraCosts,OtherCosts,Deposit,RedemptionAgreement,DateFree,...,OnlineSince,Adress,Details,LookingFor,AmountofIcons,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,10m² Zimmer in entspannter 4er Wg :),10m²,690€,690€,n.a.,n.a.,1500€,500€,01.06.2024,...,40 Minuten,Joergstraße 84\r\n ...,10m²\r\n \r...,Geschlecht egal\r\n ...,6,4,Hallo du:) \r\n Dein neues Zimmer ist angenehm...,Unsere WG ist circa 8 Gehminuten vom Laimer Pl...,"Wir sind Francesco, Dan und Kibo :) Unsere Lie...","Erzähl mir gern etwas über dich, sodass wir sc..."
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,10qm Zimmer in Doppelhaushälfte,10m²,690€,590€,100€,0€,900€,0€,01.08.2024,...,45 Minuten,gleißmüllerstr. 9\r\n ...,10m²\r\n \r...,Mann\r\n ...,11,2,Unsere WG hat 6 Zimmer im Grünen und liegt an ...,n.a.,n.a.,n.a.
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Helles Zimmer in 4-er WG,17m²,755€,755€,n.a.,n.a.,n.a.,n.a.,01.07.2024,...,45 Minuten,Holsteiner Straße\r\n ...,17m²\r\n \r...,Geschlecht egal\r\n,5,3,"Hey, \r\n leider muss ich Ende Juni aus meiner...",Die Wohnung befindet sich direkt am Südpark un...,"Die WG ist nicht als Zweck-Wg angedacht, sonde...",Im Zimmer steht ein schöner und geräumiger Kle...
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,2er Studenten WG. Wohnen neben der Uni. Netter...,10m²,850€,750€,100€,n.a.,n.a.,0€,15.06.2024,...,46 Minuten,Schellingstraße 21\r\n ...,10m²\r\n \r...,Geschlecht egal\r\n ...,2,n.a.,Bitte beachte: \r\n Das Zimmer ist rein von de...,In einem Umkreis von ca. 200m Luftlinie hast d...,"Was ich auf keinen Fall will ist ne Zweck-WG, ...",Schreib mir einfach über wg-gesucht ein paar Z...
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,103m²,1160€,1160€,n.a.,n.a.,1000€,n.a.,16.05.2024,...,46 Minuten,Heßstraße\r\n ...,103m²\r\n \...,Geschlecht egal\r\n,4,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Großes Zimmer im Zentrum in einer 3er WG direk...,20m²,1100€,1100€,0€,0€,3000€,0€,01.07.2024,...,2 Tage,Barerstraße 42\r\n ...,20m²\r\n \r...,Geschlecht egal\r\n,4,4,Das Zimmer in der Wohnung ist möbiliert mit Be...,n.a.,n.a.,n.a.
496,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,"Groß, zentral, ruhig - ZIMMER in WG am Goethep...",20m²,1150€,1050€,100€,n.a.,2000€,n.a.,01.06.2024,...,2 Tage,Waltherstr.\r\n ...,20m²\r\n \r...,Frau\r\n ...,10,4,Neu renoviertes und möbliertes WG - Zimmer in ...,"Ruhige Seitenstraße am Goetheplatz, 100m zur U...",Ausstattung \r\n Hochwertig möbliertes Zimmer ...,Mindestmietzeit 6 Monate \r\n\r\n Modernisiert...
497,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Mitbewohnerin gesucht - WG-Zimmer 16m² + eigen...,16m²,900€,900€,0€,0€,n.a.,0€,27.05.2024,...,2 Tage,Schäufeleinstr. 32\r\n ...,16m²\r\n \r...,Frau\r\n,9,4,Die Wohnung befindet sich im 3. und 4. Stock e...,Die Wohnung befindet sich in Laim und ist ca. ...,"Dein zukünftiger Mitbewohner ist Miguel, er is...",n.a.
498,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Modernes und gemütli...,9m²,1140€,1140€,n.a.,n.a.,n.a.,n.a.,05.08.2024,...,8 Stunden,Wittelsbacherstraße\r\n ...,9m²\r\n \r\...,Geschlecht egal\r\n,4,n.a.,- Handtücher und Bettlaken \r\n - Hygieneutens...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: MUC-768972
